In [1]:
import omop_alchemy as oa
from omop_alchemy.db import Base, engine
from omop_alchemy.helpers.create_db import create_db 
from omop_alchemy.helpers.populate_db import populate_demo_db, to_load_vocabulary, to_load_clinical, to_load_health_system

In [2]:
# this script will create a new sqlite db file with the tables to reflect the specified objects in 
# clinical and reference directories. needs to be made more friendly for re-create / update etc, but 
# will do for now :)

create_db(Base, engine)

In [3]:
# make sure to download a small athena zip file (suggested no more than 2-4 vocabs for demo purposes)
# and unzip to omop_alchemy/resources/ohdsi_vocabs before running this script. 

populate_demo_db(to_load_health_system)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: CARE_SITE.csv
complete load for: LOCATION.csv
complete load for: PROVIDER.csv


In [4]:

populate_demo_db(to_load_clinical)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: PERSON.csv
complete load for: CONDITION_OCCURRENCE.csv
complete load for: MEASUREMENT.csv


In [5]:
populate_demo_db(to_load_vocabulary)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/ohdsi_vocabs
complete load for: VOCABULARY.csv
complete load for: CONCEPT.csv
complete load for: CONCEPT_CLASS.csv
complete load for: DOMAIN.csv
complete load for: RELATIONSHIP.csv


In [15]:

import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss
from omop_alchemy.db.config import engine, config
from omop_alchemy.tables.clinical import Person, Condition_Occurrence, Measurement

with so.Session(engine) as sess:
    people = sess.query(Person
                        ).options(so.joinedload(Person.conditions, 
                                                Condition_Occurrence.condition_concept)
                        ).options(so.joinedload(Person.conditions,
                                                Condition_Occurrence.modifiers,
                                                Measurement.measurement_concept)).all()

In [18]:
[[(c.condition_concept.concept_name, [m.measurement_concept for m in c.modifiers]) for c in p.conditions] for p in people]

[[('Adenocarcinoma, NOS, of prostate gland',
   [<Concept 35918328 - 440 (Grade)>,
    <Concept 35918383 - 960 (TNM Clin M)>,
    None])],
 [('Neoplasm, malignant of breast, NOS', [])],
 [('Spindle cell sarcoma of lesser curvature of stomach, NOS', [])],
 [('Giant cell carcinoma of descending colon', []),
  ('Neoplasm, metastatic of liver', [])]]

In [ ]:
import csv
from pathlib import Path
from datetime import datetime

import sqlalchemy as sa
import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss

from omop_alchemy.db.config import engine, config
from omop_alchemy.tables.clinical import Person, Condition_Occurrence, Measurement
from omop_alchemy.tables.health_system import Care_Site, Location, Provider
from omop_alchemy.tables.vocabulary import Concept, Vocabulary, Concept_Class, Domain, Relationship, \
                                Concept_Relationship, Concept_Ancestor

# TODO: insert some validation and checks to make sure folk know how and why to do this (and the limits for demo purposes)

to_load_vocabulary = {'folder': 'ohdsi_vocabs',
                      'VOCABULARY.csv': Vocabulary, 
                      'CONCEPT.csv': Concept, 
                      'CONCEPT_CLASS.csv': Concept_Class, 
                      'DOMAIN.csv': Domain,
                      'RELATIONSHIP.csv': Relationship}
                      #'CONCEPT_RELATIONSHIP.csv': Concept_Relationship,
                      #'CONCEPT_ANCESTOR.csv': Concept_Ancestor}

to_load_health_system = {'folder': 'demo_data',
                         'CARE_SITE.csv': Care_Site,
                         'LOCATION.csv': Location,
                         'PROVIDER.csv': Provider}

to_load_clinical = {'folder': 'demo_data',
                    'PERSON.csv': Person,
                    'CONDITION_OCCURRENCE.csv': Condition_Occurrence,
                    'MEASUREMENT.csv': Measurement}#,
                    #'CONCEPT.csv': Concept}

# flexible loading of ohdsi vocab files downloaded to the path /data/ohdsi_vocabs

def datetime_conversion(dt, fmt):
    if dt != '':
        return datetime.strptime(dt, fmt)
    
def convert_date_col(dt):
    return datetime_conversion(dt, '%Y%m%d')
    
def convert_time_col(dt):
    return datetime_conversion(dt, '%H%M%S')

def convert_datetime_col(dt):
    return datetime_conversion(dt, '%Y%m%d%H%M%S')

def callable_pass(s):
    return s

def convert_int(i):
    try:
        return int(i)
    except:
        return 0
    
def convert_dec(i):
    try:
        return Decimal(i)
    except:
        return 0

type_map = {sss.BigInteger: convert_int, 
            sss.Integer: convert_int, 
            sss.Numeric: convert_dec, 
            sss.DateTime: convert_datetime_col, 
            sss.Time: convert_time_col, 
            sss.String: callable_pass, 
            sss.Date: convert_date_col}

def get_type_lookup(interface):
    return {c.key: type_map[type(c.type)] for c in interface.__table__._columns}



to_load = to_load_clinical

with so.Session(engine) as sess:
    folder = Path(config.VOCAB_PATH) / to_load['folder']
    print(folder)
    for ohdsi_file, interface in to_load.items():
        if interface != folder.name:
                with open(folder / ohdsi_file, 'r') as file:
                    reader = csv.DictReader(file, delimiter='\t')
                    field_map = get_type_lookup(interface)
                    
                    for row in reader:
                        record = {field:field_map[field](data) for field, data in row.items() if field in field_map}
                        o = interface(**record)
                        sess.add(o)

                    print(f'complete load for: {ohdsi_file}')
            
    sess.commit()


In [ ]:
for m in Person.registry.mappers:
    print(m.class_)
    print('\t', m.c.keys())

In [ ]:
Person.gender_concept.expression